In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
# reading the page rank text file
df = sc.textFile("/content/pagerank.txt")

In [ ]:
df.collect()

['4 5',
 '5 8',
 '8 2',
 '8 8',
 '9 4',
 '9 1',
 '7 2',
 '5 9',
 '5 5',
 '7 7',
 '2 7',
 '4 1',
 '6 6',
 '3 8',
 '9 9',
 '4 8',
 '5 5',
 '2 4',
 '4 3',
 '1 8',
 '6 4',
 '8 4',
 '6 2',
 '3 8',
 '7 1',
 '9 6',
 '9 8',
 '2 7',
 '3 7',
 '4 9',
 '1 4',
 '3 4',
 '1 3',
 '3 6',
 '2 5',
 '1 2',
 '5 6',
 '4 6',
 '1 1',
 '6 4',
 '5 1',
 '8 6',
 '3 4',
 '9 9',
 '5 6',
 '4 5',
 '6 6',
 '6 7',
 '5 2',
 '6 8']

In [ ]:
edges = df.map(lambda line: tuple(map(int, line.strip().split())))

In [ ]:
edges.collect()

[(4, 5),
 (5, 8),
 (8, 2),
 (8, 8),
 (9, 4),
 (9, 1),
 (7, 2),
 (5, 9),
 (5, 5),
 (7, 7),
 (2, 7),
 (4, 1),
 (6, 6),
 (3, 8),
 (9, 9),
 (4, 8),
 (5, 5),
 (2, 4),
 (4, 3),
 (1, 8),
 (6, 4),
 (8, 4),
 (6, 2),
 (3, 8),
 (7, 1),
 (9, 6),
 (9, 8),
 (2, 7),
 (3, 7),
 (4, 9),
 (1, 4),
 (3, 4),
 (1, 3),
 (3, 6),
 (2, 5),
 (1, 2),
 (5, 6),
 (4, 6),
 (1, 1),
 (6, 4),
 (5, 1),
 (8, 6),
 (3, 4),
 (9, 9),
 (5, 6),
 (4, 5),
 (6, 6),
 (6, 7),
 (5, 2),
 (6, 8)]

Question 1

In [ ]:
grouped_edges = edges.groupByKey()

In [ ]:
grouped_incoming_edges = grouped_edges.map(lambda x:(x[0],sorted(list(x[1]))))

In [ ]:
grouped_incoming_edges.collect()

[(4, [1, 3, 5, 5, 6, 8, 9]),
 (8, [2, 4, 6, 8]),
 (2, [4, 5, 7, 7]),
 (6, [2, 4, 4, 6, 6, 7, 8]),
 (5, [1, 2, 5, 5, 6, 6, 8, 9]),
 (9, [1, 4, 6, 8, 9, 9]),
 (7, [1, 2, 7]),
 (3, [4, 4, 6, 7, 8, 8]),
 (1, [1, 2, 3, 4, 8])]

In [ ]:
Sorted_incoming_edges = grouped_incoming_edges.sortByKey()

In [ ]:
# 1) Incoming edges sorted based on the node id
Sorted_incoming_edges.collect()

[(1, [1, 2, 3, 4, 8]),
 (2, [4, 5, 7, 7]),
 (3, [4, 4, 6, 7, 8, 8]),
 (4, [1, 3, 5, 5, 6, 8, 9]),
 (5, [1, 2, 5, 5, 6, 6, 8, 9]),
 (6, [2, 4, 4, 6, 6, 7, 8]),
 (7, [1, 2, 7]),
 (8, [2, 4, 6, 8]),
 (9, [1, 4, 6, 8, 9, 9])]

Question 2

In [ ]:
incoming_5_edges = Sorted_incoming_edges.filter(lambda x: len(x[1]) == 5)

In [ ]:
incoming_5_edges.collect()

[(1, [1, 2, 3, 4, 8])]

In [ ]:
# 2) Node that has 5 incoming edges
incoming_5_edges.map(lambda x: x[0]).collect()

[1]

Question 3

In [ ]:
node_1 = Sorted_incoming_edges.filter(lambda x: x[0] == 1)

In [ ]:
node_1_edges = node_1.map(lambda x: x[1])

In [ ]:
# 3) Incoming edges of Node 1
node_1_edges.collect()

[[1, 2, 3, 4, 8]]